<a href="https://colab.research.google.com/github/JoaoEmanuel14/ufs-ia-trabalho-a2/blob/main/IA_Trabalho_A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explorando Modelos Multimodais: Como Texto, Áudio e Imagem se Combinam em IA

Repositório no GitHub: https://github.com/JoaoEmanuel14/ufs-ia-trabalho-a2

Dúvidas? Contate-nos em: joao.apostolo@dcomp.ufs.br

In [15]:
# Configurações de sistema utilizadas para fazer os experimentos
!nvidia-smi

Tue Apr  8 12:28:24 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   66C    P0             30W /   70W |     612MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 1. Instalação de bibliotecas, imports e chaves das APIs

In [16]:
# Instalar bibliotecas necessárias para texto
!pip install openai google-generativeai -q
!pip install sentence-transformers

import openai
import google.generativeai as genai
from google.colab import userdata  # Para acessar as chaves armazenadas no Secrets
from sentence_transformers import SentenceTransformer, util

# Instalar bibliotecas necessárias para imagem
import torch
from PIL import Image
from IPython.display import display

# Instalar bibliotecas necessárias para áudio
!pip install openai vosk gtts -q
!pip install pydub

import wave
import time
import json
import io
import os
import base64
import pydub
import urllib.request
import zipfile
from gtts import gTTS
from glob import glob
from vosk import Model, KaldiRecognizer
from IPython.display import Audio
from pydub import AudioSegment

# Pegar conteúdo do Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Configurar chaves de API usando Secrets do Colab
try:
  openai_api_key = userdata.get('OPENAI_API_KEY')  # Chave da OpenAI
  google_api_key = userdata.get('GOOGLE_API_KEY')  # Chave do Google
  genai.configure(api_key=google_api_key)
except Exception as e:
  print("Erro ao carregar as chaves de API. Verifique se as chaves estão configuradas corretamente no Secrets.")
  raise e

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. Configuração dos clientes OpenAI e Google

### 2.1. Funções que processam áudio e imagem, gerando texto

In [17]:
# Configurar o cliente da OpenAI
client = openai.OpenAI(api_key=openai_api_key)

# Função para chamada ao GPT-4o com imagem e texto
def gerar_texto_openai(imagem_path, texto):
  try:
    with open(imagem_path, "rb") as image_file:
      # Utilizando a biblioteca base64 para fazer a leitura da imagem
      imagem_base = base64.b64encode(image_file.read()).decode("utf-8")
      # É necessário transformar a imagem em URL para que o ChatGPT possa lê-la
      imagem_url = f"data:image/jpeg;base64,{imagem_base}"

      # Completion para o prompt
      response = client.chat.completions.create(
          model="gpt-4o",
          # Montando a mensagem
          messages=[
              {"role": "user", "content": [
                {"type": "text", "text": texto},
                {"type": "image_url", "image_url": {"url": imagem_url}}
              ]}
          ]
      )
      return response.choices[0].message.content

  except Exception as e:
    return f"Erro ao gerar texto com OpenAI: {str(e)}"

# Função para chamada ao Gemini com imagem e texto
def gerar_texto_google(imagem_path, texto):
  try:
    model = genai.GenerativeModel("gemini-1.5-pro")
    imagem = Image.open(imagem_path)
    # Completion para o prompt
    response = model.generate_content([texto, imagem])
    return response.text

  except Exception as e:
    return f"Erro ao gerar texto com Google Gemini: {str(e)}"

### 2.2. Text-To-Speech (TTS): funções de conversão $texto \implies áudio$

In [18]:
#Função para conversão de texto em fala (TTS) com gTTS
def tts_gtts(texto):
  try:
    # Criar o objeto gTTS
    tts = gTTS(texto, lang="pt")  # Define o idioma como pt-BR
    # Salvar o áudio em um arquivo MP3
    tts.save("gtts_audio.mp3")
    print("Áudio gerado e salvo como 'gtts_audio.mp3'.")
    return "gtts_audio.mp3"

  except Exception as e:
    print(f"Erro ao gerar áudio com gTTS: {str(e)}")
    return None

# Configurar o cliente da OpenAI
client = openai.OpenAI(api_key=openai_api_key)

# Função para conversão de texto em fala (TTS) com OpenAI
def tts_openai(texto):
  try:
    # Completion para o prompt
    response = client.audio.speech.create(
        model="tts-1",
        input=texto,
        voice="alloy"
    )
    return response.content  # Retorna o conteúdo de áudio diretamente

  except Exception as e:
    print(f"Erro ao gerar áudio com OpenAI TTS: {str(e)}")
    return None

### 2.3. Função auxiliar que converte áudio do formato `mp3` para `wav`

In [19]:
# Função que converte mp3 para wav
def converter_mp3_para_wav(mp3_path, wav_path="audio.wav"):
  try:
    audio = AudioSegment.from_mp3(mp3_path)
    # Colocando o áudio no padrao PCM 16kHz mono, 16-bit
    audio = audio.set_frame_rate(16000).set_channels(1).set_sample_width(2)
    # Exportando o áudio no formato correto
    audio.export(wav_path, format="wav")
    print(f"Conversão concluída: '{wav_path}'")
    return wav_path

  except Exception as e:
    print(f"Erro na conversão de MP3 para WAV: {e}")
    return None

### 2.4. Função para transcrição de áudio usando Vosk

In [20]:
# Transcrever áudio usando Vosk
def transcrever_audio(caminho_audio):
  try:
    # Verifica se o modelo de reconhecimento de voz já foi baixado
    if not os.path.exists("vosk-model-small-pt-0.3"):
      print("Baixando o modelo...")
      # Faz o download do modelo de voz em português
      urllib.request.urlretrieve(
          "https://alphacephei.com/vosk/models/vosk-model-small-pt-0.3.zip",
          "vosk-model-small-pt-0.3.zip"
      )
      # Extrai os arquivos do modelo compactado
      with zipfile.ZipFile("vosk-model-small-pt-0.3.zip", "r") as zip_ref:
        zip_ref.extractall(".")

    wf = wave.open(caminho_audio, "rb")

    # Verifica se o áudio está no formato correto (mono, 16-bit, 16kHz)
    if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getframerate() != 16000:
      print("Formato de áudio inválido para Vosk. Esperado: WAV 16kHz mono PCM.")
      return None

    # Carrega o modelo Vosk
    model = Model("vosk-model-small-pt-0.3")

    # Inicializa o reconhecedor com o modelo e a taxa de amostragem do áudio
    rec = KaldiRecognizer(model, wf.getframerate())
    results = []

    # Lê o áudio em blocos e processa com o reconhecedor
    while True:
      data = wf.readframes(4000) # Lê 4000 frames de cada vez
      if len(data) == 0:
        break
      # Se a transcrição parcial for feita
      if rec.AcceptWaveform(data):
        # Converte o resultado para dicionário
        result = json.loads(rec.Result())
        # Adiciona o texto transcrito à lista
        results.append(result.get("text", ""))

    # Adiciona a transcrição final (último trecho processado)
    final = json.loads(rec.FinalResult())
    results.append(final.get("text", ""))

    # Junta todos os trechos de texto em uma única string
    texto = " ".join(results).strip()

    # Retorna o texto final, caso este nao esteja vazio
    if texto:
      return texto
    # Se estiver vazio, é retornado None
    else:
      return None

  except Exception as e:
    print("Erro na transcrição de áudio:", e)
    return None

### 2.5. Manipulação do arquivo que será usado para a construção da narrativa


In [21]:
# Função responsável por gerar os a tupla de pares (imagem, áudio)
def gerar_pares_imagem_audio(pasta_imagens, pasta_audios_mp3):
  pares_imagem_audio = []

  # Ordena os arquivos para garantir a correspondência sequencial
  imagens = sorted(glob(os.path.join(pasta_imagens, "*.jpg")))
  audios_mp3 = sorted(glob(os.path.join(pasta_audios_mp3, "*.mp3")))

  # O processo é interrompido caso o tamanho da pasta de imagens e de áudios for diferente
  if len(imagens) != len(audios_mp3):
    print("Quantidade de imagens e áudios não bate!")
    return []

  # Gera os pares após converter o áudio para o formato correto
  for idx, imagem in enumerate(imagens):
    audio_mp3 = audios_mp3[idx]

    wav_path = f"audio_convertido_{idx + 1}.wav"
    # Converte os áudios de .mp3 para .wav
    audio_wav_convertido = converter_mp3_para_wav(audio_mp3, wav_path)

    if audio_wav_convertido:
      pares_imagem_audio.append((imagem, audio_wav_convertido))
    else:
      print(f"[{idx + 1}] Erro ao converter áudio {audio_mp3}.")

  return pares_imagem_audio

### 2.6. Acurácia com similaridade semântica

In [22]:
# Inicializar modelo de embeddings
modelo = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')  # Uso de um modelo multilíngue

def calcular_similaridade_semantica(texto_esperado, texto_gerado):
  embedding1 = modelo.encode(texto_esperado, convert_to_tensor=True)
  embedding2 = modelo.encode(texto_gerado, convert_to_tensor=True)

  similaridade = util.pytorch_cos_sim(embedding1, embedding2).item()
  return round(similaridade, 3)

def avaliar_acuracia_multimodal(descricao_esperada, descricao_gerada):
  similaridade = calcular_similaridade_semantica(descricao_esperada, descricao_gerada)
  print(f"Similaridade semântica: {similaridade}")
  return similaridade

## 3. Entrada Multimodal

Fornecer uma imagem e um áudio descrevendo a cena, e pedir ao modelo para gerar um resumo textual.

In [23]:
# Função para limitar a entrada de texto transcrita
def limitar_texto(texto, limite=16000):
  if len(texto) > limite:
    return texto[:limite] + "..."

  return texto

# Entrada multimodal
def entrada_multimodal(caminho_imagem, caminho_audio_mp3):
  # Conversão do áudio para o formato correto
  caminho_wav = converter_mp3_para_wav(caminho_audio_mp3)
  if not caminho_wav:
    print("Erro ao converter áudio")
    return

  # Gera o texto transcrito a partir do áudio
  texto_transcrito = limitar_texto(transcrever_audio(caminho_wav))
  if not texto_transcrito:
    print("Erro: Transcrição falhou!")
    return

  prompt = f"Aqui está uma descrição de áudio: {texto_transcrito}. Com base nesta descrição e na imagem fornecida, gere um resumo textual."

  inicio_chatgpt = time.time()
  resposta_chatgpt = gerar_texto_openai(caminho_imagem, prompt)
  fim_chatgpt = time.time()
  tempo_chatgpt = fim_chatgpt - inicio_chatgpt

  inicio_gemini = time.time()
  resposta_gemini = gerar_texto_google(caminho_imagem, prompt)
  fim_gemini = time.time()
  tempo_gemini = fim_gemini - inicio_gemini

  # Acurácia multimodal
  acuracia_chatgpt = calcular_similaridade_semantica(texto_transcrito, resposta_chatgpt)
  acuracia_gemini = calcular_similaridade_semantica(texto_transcrito, resposta_gemini)

  # Resultados
  print("\n--- Resultados ---")
  print("Texto transcrito:", texto_transcrito)
  print("\nChatGPT:")
  print("Resposta:", resposta_chatgpt)
  print(f"Tempo de resposta: {tempo_chatgpt:.2f} segundos")
  print(f"Acurácia (similaridade): {acuracia_chatgpt}")

  print("\nGemini:")
  print("Resposta:", resposta_gemini)
  print(f"Tempo de resposta: {tempo_gemini:.2f} segundos")
  print(f"Acurácia (similaridade): {acuracia_gemini}")

In [24]:
sentenca = "Uma praia bonita. Possui coqueiros e água cristalina."
audio_gtts = tts_gtts(sentenca)
praia = '/content/drive/MyDrive/Inteligência Artificial/praia.jpg'
entrada_multimodal(praia, audio_gtts)

Áudio gerado e salvo como 'gtts_audio.mp3'.
Conversão concluída: 'audio.wav'

--- Resultados ---
Texto transcrito: uma praia bonita possui coqueiros e água cristalina

ChatGPT:
Resposta: A imagem retrata uma praia paradisíaca com areia clara e fina, cercada por coqueiros que criam uma sombra relaxante. A água é cristalina e de um azul vibrante, misturando-se com o céu claro e quase sem nuvens. Ao fundo, veem-se colinas cobertas por vegetação exuberante, contribuindo para um cenário tropical e sereno.
Tempo de resposta: 4.16 segundos
Acurácia (similaridade): 0.855

Gemini:
Resposta: Uma serena praia tropical é retratada, com palmeiras inclinadas lançando sombras suaves na areia branca e primitiva. Águas cristalinas e turquesa lambem suavemente a costa, encontrando a exuberante vegetação verde de uma colina ao longe. O céu está pintado com um azul brilhante, pontilhado de nuvens fofas e brancas.
Tempo de resposta: 3.94 segundos
Acurácia (similaridade): 0.803


## 4. Transferência de modalidade
Fornecer apenas a imagem e pedir ao modelo para gerar um áudio descritivo.

In [25]:
# Transferência de modalidade
def transferencia_modalidade(imagem_path, descricao_referencia):
  prompt = "Descreva esta imagem minuciosamente para que esta possa ser convertida em formato de áudio"

  print("Gerando descrição do ChatGPT...")
  inicio_chatgpt = time.time()
  descricao_chatgpt = gerar_texto_openai(imagem_path, prompt)
  fim_chatgpt = time.time()
  tempo_chatgpt = fim_chatgpt - inicio_chatgpt

  # Geração do áudio do ChatGPT a partir da descrição
  if descricao_chatgpt and not descricao_chatgpt.startswith("Erro"):
    print(f"Descrição gerada para áudio pelo ChatGPT: \n{descricao_chatgpt}")
    print(f"Tempo de resposta do ChatGPT: {tempo_chatgpt:.2f} segundos")

    # Acurácia multimodal
    acuracia_chatgpt = calcular_similaridade_semantica(descricao_referencia, descricao_chatgpt)
    print(f"Acurácia (ChatGPT): {acuracia_chatgpt}")

    # Geração e reprodução do áudio
    audio_chatgpt = tts_openai(descricao_chatgpt)
    if audio_chatgpt:
      with open("descricao_chatgpt.mp3", "wb") as f:
        # Salvando o áudio
        f.write(audio_chatgpt)
      print("Áudio salvo como 'descricao_chatgpt.mp3'")
      display(Audio("descricao_chatgpt.mp3", autoplay=True))
  else:
    print(f"Erro ao gerar descrição: {descricao_chatgpt}")

  print("\nGerando descrição do Gemini...")
  inicio_gemini = time.time()
  descricao_gemini = gerar_texto_google(imagem_path, prompt)
  fim_gemini = time.time()
  tempo_gemini = fim_gemini - inicio_gemini

  # Geração do áudio do Gemini a partir da descrição
  if descricao_gemini and not descricao_gemini.startswith("Erro"):
    print(f"Descrição gerada para áudio pelo Gemini: \n{descricao_gemini}")
    print(f"Tempo de resposta do Gemini: {tempo_gemini:.2f} segundos")

    # Acurácia multimodal
    acuracia_gemini = calcular_similaridade_semantica(descricao_referencia, descricao_gemini)
    print(f"Acurácia (Gemini): {acuracia_gemini}")

    # Geração e reprodução do áudio
    audio_gemini = tts_openai(descricao_gemini)
    if audio_gemini:
      with open("descricao_gemini.mp3", "wb") as f:
        # Salvando o áudio
        f.write(audio_gemini)
      print("Áudio salvo como 'descricao_gemini.mp3'")
      display(Audio("descricao_gemini.mp3", autoplay=True))
  else:
    print(f"Erro ao gerar descrição: {descricao_gemini}")

  # Resumo final
  print("\n--- Resumo dos tempos e acurácias ---")
  print(f"- ChatGPT: {tempo_chatgpt:.2f}s | Acurácia: {acuracia_chatgpt}")
  print(f"- Gemini:  {tempo_gemini:.2f}s | Acurácia: {acuracia_gemini}")

In [26]:
praia = '/content/drive/MyDrive/Inteligência Artificial/praia.jpg'
transferencia_modalidade(praia, "Uma praia bonita. Possui coqueiros e água cristalina.")

Gerando descrição do ChatGPT...
Descrição gerada para áudio pelo ChatGPT: 
A imagem retrata uma praia tropical deslumbrante. No primeiro plano, há uma série de palmeiras esguias, cujas folhas verdes inclinam-se em direção à praia, criando sombras sobre a areia. A areia é clara e fina, estendendo-se em direção ao mar. À direita, o mar apresenta um degradê de tons azul-turquesa, com pequenas ondas espumando suavemente na margem. O céu está claro, com algumas nuvens brancas dispersas, e ao fundo, uma colina coberta por vegetação densa e tropical se ergue suavemente, completando o cenário paradisíaco.
Tempo de resposta do ChatGPT: 4.29 segundos
Acurácia (ChatGPT): 0.803
Áudio salvo como 'descricao_chatgpt.mp3'



Gerando descrição do Gemini...
Descrição gerada para áudio pelo Gemini: 
A imagem apresenta uma cena tropical idílica, semelhante a um paraíso. O primeiro plano é dominado por uma praia de areia branca e imaculada, que se estende para o quadro, convidando o espectador a dar um passeio. As sombras irregulares dos imponentes coqueiros projetam-se sobre a areia, acrescentando profundidade e textura.

O canto inferior esquerdo é dominado por uma densa coleção de coqueiros, seus troncos inclinando-se graciosamente em direção ao oceano. Suas folhas verde-esmeralda vibrantes criam um contraste vibrante com a areia branca e o azul turquesa da água.

O mar ocupa a parte central e direita da imagem, exibindo uma deslumbrante gama de tons azuis, do turquesa claro próximo à costa até um azul profundo mais adiante. Pequenas ondas suaves quebram suavemente na praia, criando uma linha branca de espuma na borda da água.

Ao fundo, uma colina verdejante e exuberante eleva-se do mar, proporcionando um 


--- Resumo dos tempos e acurácias ---
- ChatGPT: 4.29s | Acurácia: 0.803
- Gemini:  8.07s | Acurácia: 0.747


## 5. Avaliação do Contexto

Fornecer uma sequência de imagens com descrições em áudio e verificar se o modelo consegue identificar a progressão da narrativa.

In [27]:
# Avaliação do contexto para OpenAI ChatGPT
def avaliacao_do_contexto_openai(pares_imagem_audio, avaliacao_referencia):
  """pares_imagem_audio: lista de tuplas (imagem_path, audio_path)
  avaliacao_referencia: string que representa a avaliacao esperada como resposta do modelo
  """
  textos_transcritos = []
  imagens_bytes = []

  print("\nTranscrevendo áudios e carregando imagens...")
  for idx, (imagem, audio) in enumerate(pares_imagem_audio):
    texto = transcrever_audio(audio)
    if texto:
      textos_transcritos.append((imagem, audio))
      try:
        with open(imagem, "rb") as imagem_file:
          imagens_bytes.append(imagem_file.read())

        print(f"[{idx + 1}] Texto transcrito: {texto}")

      except Exception as e:
        print(f"[{idx + 1}] Erro ao ler a imagem: {e}")
        return
    else:
      print(f"[{[idx + 1]}] Falha ao transcrever o áudio: {audio}")
      return

  print("\nAnalisando progressão narrativa com ChatGPT")
  inicio_chatgpt = time.time()

  # Mensagem inicial do prompt
  mensagens = [{"role": "system", "content": "Você é um especialista em análise narrativa. Avalie se há progressão lógica entre as imagens e as descrições fornecidas."}]

  # Montar a sequência multimodal para enviar ao modelo
  for idx, (imagem_bytes, texto) in enumerate(zip(imagens_bytes, textos_transcritos)):
    mensagens.append({
        "role": "user",
        "content": [
            {"type": "text", "text": f"Imagem {idx + 1}: {texto}"},
            {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64.b64encode(imagem_bytes).decode('utf-8')}"}}
        ]
    })

  # Pergunta final
  mensagens.append({
      "role": "user",
      "content": "Com base nas imagens e nas descrições anteriores, existe uma narrativa progressiva?"
  })

  try:
    # Completion para o prompt
    response = client.chat.completions.create(
        model = "gpt-4o",
        messages=mensagens
    )
    resultado = response.choices[0].message.content
    fim_chatgpt = time.time()
    tempo_chatgpt = fim_chatgpt - inicio_chatgpt

    print("Avaliação do contexto (ChatGPT):")
    print(resultado)
    print(f"Tempo de resposta do ChatGPT: {tempo_chatgpt:.2f} segundos")

    # Acurácia multimodal
    acuracia_gpt = calcular_similaridade_semantica(avaliacao_referencia, resultado)
    print(f"Acurácia multimodal (ChatGPT): {acuracia_gpt}")

    # return resultado

  except Exception as e:
    print(f"Erro ao analisar a narrativa: {e}")
    return None

# Avaliação do contexto para Google Gemini
def avaliacao_do_contexto_google(pares_imagem_audio, avaliacao_referencia):
  """pares_imagem_audio: lista de tuplas (imagem_path, audio_path)
  avaliacao_referencia: string que representa a avaliacao esperada como resposta do modelo
  """
  textos_transcritos = []
  imagens_bytes = []

  print("\nTranscrevendo áudios e carregando imagens...")
  for idx, (imagem, audio) in enumerate(pares_imagem_audio):
    texto = transcrever_audio(audio)
    if texto:
      textos_transcritos.append((imagem, audio))
      try:
        with open(imagem, "rb") as imagem_file:
          imagens_bytes.append(imagem_file.read())

        print(f"[{idx + 1}] Texto transcrito: {texto}")

      except Exception as e:
        print(f"[{idx + 1}] Erro ao ler a imagem: {e}")
        return
    else:
      print(f"[{[idx + 1]}] Falha ao transcrever o áudio: {audio}")
      return

  print("\nAnalisando progressão narrativa com Gemini")
  inicio_gemini = time.time()

  try:
    # Carrega o modelo do Google Gemini
    model = genai.GenerativeModel("gemini-1.5-pro")

  except Exception as e:
    print(f"Erro ao inicializar o modelo Google Gemini: {str(e)}")
    return

  # Monta a entrada multimodal
  partes_entrada = [
      "Você é um especialista em análise narrativa. Avalie a sequência de imagens e descrições fornecidas.",
      "Determine se existe uma progressão narrativa lógica entre os eventos.\n"
  ]

  # Coloca os pares (imagem, áudio) no prompt
  for idx, (imagem_bytes, texto) in enumerate(zip(imagens_bytes, textos_transcritos)):
    partes_entrada.append(f"Imagem {idx + 1}: {texto}")
    partes_entrada.append({"mime_type": "image/jpeg", "data": imagem_bytes})

  # Pergunta final
  partes_entrada.append("Com base nas imagens e nas descrições anteriores, existe uma narrativa progressiva?")

  try:
    # Completion para o modelo
    resultado = model.generate_content(partes_entrada)
    fim_gemini = time.time()
    tempo_gemini = fim_gemini - inicio_gemini

    print("Avaliação do contexto (Gemini):")
    print(resultado.text)
    print(f"Tempo de resposta do Gemini: {tempo_gemini:.2f} segundos")

    # Acurácia multimodal
    acuracia_gemini = calcular_similaridade_semantica(avaliacao_referencia, resultado.text)
    print(f"Acurácia multimodal (Gemini): {acuracia_gemini}\n ")

    # return resultado.text

  except Exception as e:
    print(f"Erro ao gerar análise com Gemini: {e}")
    return None

In [28]:
# Carregando as imagens e áudios do Google Drive
pasta_imagens = '/content/drive/MyDrive/Inteligência Artificial/Imagens_narrativa'
pasta_audios = '/content/drive/MyDrive/Inteligência Artificial/Audios_narrativa'
pares_imagens_audios = gerar_pares_imagem_audio(pasta_imagens, pasta_audios)

narrativa = "Sim, existe uma narrativa. Ela é a seguinte: Num dia em que o sol brilhava com força, um gato cinza chamado Tobias decidiu sair para explorar o bairro. Ele andava como se tivesse um compromisso sério — cauda erguida, passos firmes. No meio do caminho, encontrou algo incomum: um guarda-chuva azul, aberto, no meio da calçada. Tobias parou. Olhou para o guarda-chuva. Depois para o céu sem nuvens. “Humano distraído”, pensou ele. Mas ao tocar no guarda-chuva com a pata, ele se fechou de repente... e um portal se abriu! Tobias foi sugado para dentro com um miau! surpreso. Quando abriu os olhos, estava numa cidade onde todos os animais usavam roupas e andavam de patinete. Um coelho de óculos escuros o olhou e disse: — Seja bem-vindo a Umbrellópolis. Aqui, só entra quem encontra o Guarda-Chuva Azul. Tobias deu um miado resignado. Aparentemente, a aventura do dia estava só começando."

avaliacao_do_contexto_openai(pares_imagens_audios, narrativa)
avaliacao_do_contexto_google(pares_imagens_audios, narrativa)

Conversão concluída: 'audio_convertido_1.wav'
Conversão concluída: 'audio_convertido_2.wav'
Conversão concluída: 'audio_convertido_3.wav'
Conversão concluída: 'audio_convertido_4.wav'
Conversão concluída: 'audio_convertido_5.wav'

Transcrevendo áudios e carregando imagens...
[1] Texto transcrito: o gato tobias está explorando o bairro
[2] Texto transcrito: o gato tobias acham guarda chuva do
[3] Texto transcrito: o guarda chuva se fecha e abrir um portal
[4] Texto transcrito: o gato tobias sugado pelo portal
[5] Texto transcrito: o gato tobias chega em um bibelô polícia recebido pelo coelho estiloso

Analisando progressão narrativa com ChatGPT
Avaliação do contexto (ChatGPT):
Sim, há uma narrativa progressiva evidente nas imagens e descrições:

1. **Imagem 1:** Um guarda-chuva se fecha e aparece um portal, sugerindo um evento mágico ou extraordinário prestes a acontecer.
2. **Imagem 2:** Tobias encontra o guarda-chuva azul, que parece ser o objeto mágico.
3. **Imagem 3:** Tobias atrave